In [ ]:
import asyncio
import logging
import sys

from lsst.ts import salobj
from lsst_efd_client import EfdClient

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
client = EfdClient("ncsa_teststand_efd")
csc_name = "ATHexapod"
csc_index = None
topic_name = f"lsst.sal.{csc_name}.logevent_summaryState"

In [ ]:
results = await client.select_top_n(topic_name, "private_origin", 1, index=csc_index)

In [ ]:
private_origin = results["private_origin"][0]

In [ ]:
d = salobj.Domain()
await asyncio.sleep(10)

In [ ]:
r = salobj.Remote(d, csc_name, index=csc_index, evt_max_history=100)
await r.start_task

In [ ]:
has_current_origin = False
try:
    while True:
        sv = await r.evt_summaryState.next(flush=False, timeout=10)
        #print(sv)
        has_current_origin = sv.private_origin == private_origin
except asyncio.TimeoutError:
    pass
finally:
    if has_current_origin:
        print("Got samples from current origin")
    else:
        print("No samples received from current origin")